In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
import sys

In [2]:
sys.setrecursionlimit(10000)

In [3]:
df = pd.read_csv('top_and_controversial_lg_wpub.csv')

In [4]:
data = []

In [6]:
for idx, row in df.iterrows():
    if idx % 100 == 0:
        print(f"{idx}/{df.shape[0]}")
        
    url = row.url 

    if data:
        if row.publisher == data[-1].publisher:
            sleep(1)
    
    try:
        page = requests.get(url, timeout=5)
        soup = BeautifulSoup(page.text, 'html.parser')
    except Exception as e:
        print(e)
        print(row)
        continue
        
    
    
#     print(page.status_code)
    if page.status_code == 200:
        try:
            row["soup"] = soup  # so I can stop scraping the damn thing
            row["p"] = soup.find_all('p')
            
            if row.title in soup.title.text:
#                 print("match title", soup.title.text)
                row["titles_match"] = True
            elif row.title in soup.h1:
#                 print(soup.h1.string)
                # check the h1 tag
                row["titles_match"] = True
            else:
#                 print(f"no match. Row title: {row.title} \nSoup title: {soup.title.text} \nH1: {soup.h1}")
                
                row["titles_match"] = False
            data.append(row)
        except Exception as e:
            print(e)
#         print("######")

0/6534
argument of type 'NoneType' is not iterable
argument of type 'NoneType' is not iterable
argument of type 'NoneType' is not iterable
'NoneType' object has no attribute 'text'
HTTPSConnectionPool(host='www.abc.net.au', port=443): Read timed out. (read timeout=5)
Unnamed: 0                                                            83
Unnamed: 0.1                                                          83
title                  Greta Thunberg catches lift to UN COP25 climat...
score                                                                  7
upvote_ratio                                                        0.52
id                                                                dvpb96
url                    https://www.abc.net.au/news/2019-11-13/austral...
comms_num                                                             55
created                                                      1.57367e+09
timestamp                                            2019-11-13 17:25:54
co

In [7]:
new_df = pd.DataFrame(data, columns=list(df.columns).extend(["soup","p","titles_match"]))

In [8]:
new_df.head()

,Unnamed: 0,Unnamed: 0.1,title,score,upvote_ratio,id,url,comms_num,created,timestamp,controversial,char_count,word_count,char_count_quantile,word_count_quantile,publisher,soup,p,titles_match
0,0,0,Elon Musk's promised ventilators never deliver...,53,0.50,g2edod,https://www.foxcarolina.com/elon-musks-promise...,396,1.587071e+09,2020-04-16 21:55:55,True,96,12,"(95.0, 96.0]","(11.0, 12.0]",www.foxcarolina.com,"[\n, html, \n, [\n, [\n, <base href=""https://w...",[[This week Elon Musk unveiled his most sci-fi...,True
2,2,2,Justin Bieber Arrested for Drag Racing / DUI (...,730,0.71,1vxp05,http://www.nbcmiami.com/news/Justin-Bieber-Arr...,3612,1.390507e+09,2014-01-23 19:48:28,True,52,9,"(51.0, 52.0]","(8.0, 9.0]",www.nbcmiami.com,"[doctype html, \n, [\n, [\n, <meta charset=""ut...",[[Pop star Justin Bieber is facing driving und...,False
5,5,5,Call for Blood Donors After Orlando Massacre E...,83,0.53,4nrsyb,http://abcnews.go.com/Health/call-blood-donors...,257,1.465788e+09,2016-06-13 04:23:57,True,76,12,"(75.0, 76.0]","(11.0, 12.0]",abcnews.go.com,"[\n, doctype html, \n, [\n, [\n, ABCNEWS | b3...",[[— -- In the wake of the Orlando shooting tha...,True
0,0,0,Elon Musk's promised ventilators never deliver...,53,0.50,g2edod,https://www.foxcarolina.com/elon-musks-promise...,396,1.587071e+09,2020-04-16 21:55:55,True,96,12,"(95.0, 96.0]","(11.0, 12.0]",www.foxcarolina.com,"[\n, html, \n, [\n, [\n, <base href=""https://w...",[[This week Elon Musk unveiled his most sci-fi...,True
2,2,2,Justin Bieber Arrested for Drag Racing / DUI (...,730,0.71,1vxp05,http://www.nbcmiami.com/news/Justin-Bieber-Arr...,3612,1.390507e+09,2014-01-23 19:48:28,True,52,9,"(51.0, 52.0]","(8.0, 9.0]",www.nbcmiami.com,"[doctype html, \n, [\n, [\n, <meta charset=""ut...",[[Pop star Justin Bieber is facing driving und...,False


In [11]:
sys.setrecursionlimit(10000)
new_df.to_csv('top_and_controversial_lg_wpub.csv')